### Scrapping Vodafone Forum

In [1]:
import requests
import re
import json
import logging
import pandas
from collections import OrderedDict
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import logging
from collections import OrderedDict
from tqdm import tqdm_notebook as tqdm
from random import seed
from random import randint
from datetime import datetime
from webdriver_manager.chrome import ChromeDriverManager
from time import time       # Other imports
from time import sleep
import csv
import time
from tqdm import tqdm_notebook as tqdm
from random import seed
from random import randint
from datetime import datetime
from selenium import webdriver
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
from time import sleep


### Drivers

In [2]:
driver = webdriver.Chrome("C:/Users/GSantos/Documents/GABRIEL RAVI/Web Scrapping/chromedriver_win32_2/chromedriver.exe")
# C:\Users\User\Documents\Masters Thesis\chromedriver_win32
# driver = webdriver.Chrome('C:/Users/ACunha/Downloads/chromedriver_win32/chromedriver.exe')

In [6]:
driver.get("https://forum.vodafone.pt/t5/forums/recentpostspage/post-type/thread/page/1")

In [4]:
content = driver.page_source
soup = BeautifulSoup(content)

In [7]:
base_url = "https://forum.vodafone.pt/"

### Scroll Down

In [71]:
from time import sleep
def scroll_down(driver) -> None:
        """Scrolling down the page and taking the URLs"""
        global listona
        global posts_url
        
        listona = []
        listona2 = []
        posts_url = []
        last_height = 0
        
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(3)
            elements = driver.find_elements_by_css_selector('.page-link.lia-link-navigation.lia-custom-event')
                       
            
            for elem in elements:
                urls = elem.get_attribute('href')
                listona.append(urls)    
                
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                break
            last_height = new_height
            
            posts_url += listona
        

           


In [9]:
scroll_down(driver)

In [10]:
print(posts_url)

['https://forum.vodafone.pt/t5/Assist%C3%AAncia-Manuais-e-Acess%C3%B3rios/Puk-perdido/m-p/400533', 'https://forum.vodafone.pt/t5/Tv-Servi%C3%A7os/N%C3%A3o-Renova%C3%A7%C3%A3o-de-Contrato-TV-NET-VOZ-Erro-na-Fideliza%C3%A7%C3%A3o-e/m-p/400528', 'https://forum.vodafone.pt/t5/Tv-Equipamentos/TV-NET-VOZ-Devolu%C3%A7%C3%A3o-de-Equipamentos/m-p/400527', 'https://forum.vodafone.pt/t5/Tv-Equipamentos/Tv-Cabos/m-p/400523', 'https://forum.vodafone.pt/t5/Samsung/Samsung-A72/m-p/400514', 'https://forum.vodafone.pt/t5/Sobre-a-Comunidade/Para-quando-a-Vodafone-na-appel-tv/m-p/400513', 'https://forum.vodafone.pt/t5/Outros-Servi%C3%A7os/Activa%C3%A7%C3%A3o-HBO-e-FOX-24-meses-OFERTA/m-p/400510', 'https://forum.vodafone.pt/t5/Yorn/Change-of-name-on-account/m-p/400508', 'https://forum.vodafone.pt/t5/Outros-Servi%C3%A7os/REEMBOLSO-CHAMADA-16913-REPARA%C3%87%C3%83O-TELEFONE-FIXO/m-p/400501', 'https://forum.vodafone.pt/t5/Outros-Servi%C3%A7os/Envio-de-SMS-ap%C3%B3s-pagamento-de-factura/m-p/400496', 'https://

In [12]:
posts_url

['https://forum.vodafone.pt/t5/Assist%C3%AAncia-Manuais-e-Acess%C3%B3rios/Puk-perdido/m-p/400533',
 'https://forum.vodafone.pt/t5/Tv-Servi%C3%A7os/N%C3%A3o-Renova%C3%A7%C3%A3o-de-Contrato-TV-NET-VOZ-Erro-na-Fideliza%C3%A7%C3%A3o-e/m-p/400528',
 'https://forum.vodafone.pt/t5/Tv-Equipamentos/TV-NET-VOZ-Devolu%C3%A7%C3%A3o-de-Equipamentos/m-p/400527',
 'https://forum.vodafone.pt/t5/Tv-Equipamentos/Tv-Cabos/m-p/400523',
 'https://forum.vodafone.pt/t5/Samsung/Samsung-A72/m-p/400514',
 'https://forum.vodafone.pt/t5/Sobre-a-Comunidade/Para-quando-a-Vodafone-na-appel-tv/m-p/400513',
 'https://forum.vodafone.pt/t5/Outros-Servi%C3%A7os/Activa%C3%A7%C3%A3o-HBO-e-FOX-24-meses-OFERTA/m-p/400510',
 'https://forum.vodafone.pt/t5/Yorn/Change-of-name-on-account/m-p/400508',
 'https://forum.vodafone.pt/t5/Outros-Servi%C3%A7os/REEMBOLSO-CHAMADA-16913-REPARA%C3%87%C3%83O-TELEFONE-FIXO/m-p/400501',
 'https://forum.vodafone.pt/t5/Outros-Servi%C3%A7os/Envio-de-SMS-ap%C3%B3s-pagamento-de-factura/m-p/400496',


In [13]:
### Do the lopp through the pages

### Coletando Informações do Post

In [14]:
# Exemplo com 3 comentários
driver.get(listona[10])

In [15]:
def info_post():
    
    global type_post_principal
    global title_post_principal
    global user_post_principal
    global class_user_principal
    global data_post_principal
    global texto_post_principal
    global likes_post_principal
    
    
    ### Category of the post
    try:
        type_post_principal = driver.find_element_by_class_name('custom-board-title').text
    except:
        pass
    if type_post_principal == ['']:
        type_post_principal = ["NaN"]   
    
    ### Title of the post
    try:
        title_post_principal = driver.find_element_by_class_name('lia-thread-subject').text
    except:
        pass
    if title_post_principal == ['']:
        title_post_principal = ["NaN"]   
    
    ### User of the post
    try:
        user_post_principal = driver.find_element_by_class_name('login-bold').text
    except:
        pass
    if user_post_principal == ['']:
        user_post_principal = ["NaN"]   
    
    ### Class of the Userthat posted
    try:
        class_user_principal = driver.find_element_by_css_selector('.lia-message-author-rank.lia-component-author-rank.lia-component-message-view-widget-author-rank').text
    except:
        pass
    if class_user_principal == ['']:
        class_user_principal = ["NaN"]   
    
    ### Data Post
    try:
        data_post_principal = driver.find_element_by_class_name('local-friendly-date').get_attribute('title').replace("\u200e","")
    except:
        pass
    if data_post_principal == ['']:
        data_post_principal = ["NaN"]   
    
    ### Post itself
    try:
        texto_post_principal = driver.find_element_by_class_name('lia-message-body-content').text.replace("\n"," ")
    except:
        pass
    if texto_post_principal == ['']:
        texto_post_principal = ["NaN"]   
    
    ### Likes of the post
    try:
        likes_post_principal = driver.find_element_by_css_selector('.MessageKudosCount.lia-component-kudos-widget-message-kudos-count').text
    except:
        pass
    if likes_post_principal == ['']:
        likes_post_principal = ["NaN"]   
    

In [82]:
info_post()
type_post_principal
title_post_principal
user_post_principal
class_user_principal
texto_post_principal
data_post_principal
texto_post_principal

'Vou aos subscrição e não cancela diz período de permanência 17abril'

### Collecting all the answers 

In [124]:
def scrap_comments():
    
    global user_posts
    global users_post2
    global class_users
    global class_users2
    global data_posts
    global data_posts2
    global texto_posts
    global texto_posts2
    global likes_posts
    global likes_post2
    
    user_posts = []
    users_post2 = []
    class_users = []
    class_users2 = []
    data_posts = []
    data_posts2 = []
    texto_posts = []
    texto_posts2 = []
    likes_post = []
    likes_post2 = []
    
    u_post = driver.find_elements_by_class_name('lia-message-author-username')
    cu_post = driver.find_elements_by_css_selector('.lia-message-author-rank.lia-component-author-rank.lia-component-message-view-widget-author-rank')
    data_post = driver.find_elements_by_class_name('local-friendly-date')
    texto_post = driver.find_elements_by_class_name('lia-message-body-content')
    likes_u_post = driver.find_elements_by_css_selector('.MessageKudosCount.lia-component-kudos-widget-message-kudos-count')
    
    
    for i in u_post:
        user_posts.append(i.text)
        
    for i in cu_post:
        class_users.append(i.text)
    
    for i in data_post:
        data_posts.append(i.get_attribute('title').replace("\u200e",""))
    data_posts = data_posts[1::2]

    for i in texto_post:
        texto_posts.append(i.text.replace("\n"," "))
   
    for i in likes_u_post:
        likes_post.append(i.text)
    
    
    users_post2 += user_posts
    class_users2 += class_users
    data_posts2 += data_posts
    texto_posts2 += texto_posts
    likes_post2 += likes_post

In [138]:
user_posts = []
users_post2 = []

In [126]:
users_post2

[]

In [127]:
scrap_comments()
print(users_post2)
# class_users2
# data_posts2
#print(texto_posts2)
# likes_post2

['carla76', 'Rute_Vodafone', 'carla76', 'Rute_Vodafone']


### Function to add the data

In [128]:
from time import time

In [139]:
start_time = time()

post_data = OrderedDict()

post_data['Tempo_recolha'] = []
post_data['Categoria'] = []
post_data['Empresa'] = []
post_data['n'] = []
post_data['URL_Post'] = []
post_data['Texto_Pub'] = []
post_data['Theme_Pub'] = []
post_data['User_Pub'] = []
post_data['Class_User_Pub'] = []
post_data['Data_Pub'] = []
post_data['Texto_Pub'] = []
post_data['Likes_Pub'] = []

post_data['User_Conv'] = []
post_data['Class_Conv'] = []
post_data['Data_Conv'] = []
post_data['Textos_Conv'] = []
post_data['Likes_Conv'] = []

post_data['N_respostas'] = []

#post_data['Autor_Post_ID'] = []

## Create the List 

In [140]:
base_url = 'https://forum.vodafone.pt/t5/forums/recentpostspage/post-type/thread/page/'
empresa= 'Vodafone'
N = 5
categoria = "Telecomunicação"

In [141]:
lista = pd.DataFrame({ 'N' : range(1, N + 1 ,1),
                      'Categoria': categoria,
                     "URL_Post": base_url,
                     "Empresa": empresa})
lista["posts"] = lista["URL_Post"] + lista["N"].astype(str) 

In [142]:
# lista["posts"][3]
lista["posts"][:1]

0    https://forum.vodafone.pt/t5/forums/recentpost...
Name: posts, dtype: object

In [143]:
lista

,N,Categoria,URL_Post,Empresa,posts
0,1,Telecomunicação,https://forum.vodafone.pt/t5/forums/recentpost...,Vodafone,https://forum.vodafone.pt/t5/forums/recentpost...
1,2,Telecomunicação,https://forum.vodafone.pt/t5/forums/recentpost...,Vodafone,https://forum.vodafone.pt/t5/forums/recentpost...
2,3,Telecomunicação,https://forum.vodafone.pt/t5/forums/recentpost...,Vodafone,https://forum.vodafone.pt/t5/forums/recentpost...
3,4,Telecomunicação,https://forum.vodafone.pt/t5/forums/recentpost...,Vodafone,https://forum.vodafone.pt/t5/forums/recentpost...
4,5,Telecomunicação,https://forum.vodafone.pt/t5/forums/recentpost...,Vodafone,https://forum.vodafone.pt/t5/forums/recentpost...


## Here comes the Magic

In [144]:
from tqdm.notebook import tqdm

In [145]:
posts_url = []

In [146]:
if __name__ == "__main__":
    
    size_ = 0
    with tqdm(total=len(lista["posts"][:1])) as pbar:
        for company, lista_posts in enumerate(lista["posts"][:1], start=0):
            
            pbar.set_description(f'Tamanho da base: {size_} obs')
            pbar.update(1)
            
            driver.get(lista_posts)
            
            scroll_down(driver)
            
            page_source = driver.page_source
            profile_bs = BeautifulSoup(page_source, 'lxml')

            company_ = lista["Empresa"][company]
            
            with tqdm(total=len(posts_url)) as pbar2:
                    for n, i in enumerate(posts_url, start=1):
                        pbar2.update(1)
                        pbar2.set_description(f'Posts da {company_}')
                                                
                        driver.get(i)
                        
                        info_post()                   
                                                
                        driver.get(i)
                        
                        user_post = []
                        class_users = []
                        data_posts = []
                        texto_posts = []
                        likes_post = []
                        
                        comment_url2 = []
                        user_post2 = []
                        class_users2 = []
                        data_posts2 = []
                        texto_posts2 = []
                        likes_post2 = []
                        
                        scrap_comments()
                                                
                           
                        while len(driver.find_elements_by_xpath("//*[contains(@aria-label,'Load more comments')]")) == 1:

                            page_source = driver.page_source
                            profile_bs = BeautifulSoup(page_source, 'lxml')

                            comment_url_aux = []

                            elements = driver.find_elements_by_css_selector('.page-link.lia-link-navigation.lia-custom-event')


                            for elem in elements:
                                urls = elem.get_attribute('href')
                                comment_url_aux.append(urls)  


                            if comment_url_aux == []:
                                break

    #                         driver.get(comment_url_aux)

    #                         scrap_comments()

                            if len(texto_posts2) == 0:
                                    users_post2, class_users2, data_posts2, texto_posts2,likes_post2 = ["NaN"], ["NaN"], ["NaN"], ["NaN"], ["NaN"]


                        comment_comment = comment_url2
                        if len(comment_url2) > 0:
                            for comment_count, aux in enumerate(comment_comment,start=0):
                                comment_url2 = []
                                driver.get(comment_comment[comment_count])
                                scrap_comments()

                                    
                        post_data['Tempo_recolha'] = np.concatenate((post_data['Tempo_recolha'], np.tile(datetime.now(), len(texto_posts2))))
                        post_data['Categoria'] = np.concatenate((post_data['Categoria'], np.tile(lista['Categoria'][company], len(texto_posts2))))
                        post_data['Empresa'] = np.concatenate((post_data['Empresa'], np.tile(lista['Empresa'][company], len(texto_posts2))))
       #                 post_data['post_author_url'] = np.concatenate((post_data['post_author_url'], np.tile(lista['URL_Post'][company], len(texto_posts2))))
                        post_data['n'] = np.concatenate((post_data['n'], np.tile(n, len(texto_posts2))))
                        post_data['URL_Post'] = np.concatenate((post_data['URL_Post'], np.tile(i, len(texto_posts2))))

                        post_data['Texto_Pub'] = np.concatenate((post_data['Texto_Pub'], np.tile(texto_post_principal, len(texto_posts2))))
                        post_data['Theme_Pub'] = np.concatenate((post_data['Theme_Pub'], np.tile(title_post_principal, len(texto_posts2))))
                        post_data['User_Pub'] = np.concatenate((post_data['User_Pub'], np.tile(user_post_principal, len(texto_posts2))))
                        post_data['Class_User_Pub'] = np.concatenate((post_data['Class_User_Pub'], np.tile(class_user_principal, len(texto_posts2))))
                        post_data['Data_Pub'] = np.concatenate((post_data['Data_Pub'], np.tile(data_post_principal, len(texto_posts2))))
                        post_data['Likes_Pub'] = np.concatenate((post_data['Likes_Pub'], np.tile(likes_post_principal, len(texto_posts2))))
                    
                    
                        size_ = len(post_data['URL_Post'])

                        post_data['User_Conv'] = post_data['User_Conv'] + users_post2
                        post_data['Class_Conv'] = post_data['Class_Conv'] + class_users2
                        post_data['Data_Conv'] = post_data['Data_Conv'] + data_posts2
                        post_data['Textos_Conv'] = post_data['Textos_Conv'] + texto_posts2
                        post_data['Likes_Conv'] = post_data['Likes_Conv'] + likes_post2
    
    

In [147]:
post_data['Texto_Pub']

array(['Boa tarde Após a compra online de uma smarphone oppo a72 não consigo activar os 15 Gb de oferta. Ao ligar o 12712 respondem-me que o meu aparelho não está abrangido pela promoção. Algém pode ajudar? Obrigado',
       'Boa tarde Após a compra online de uma smarphone oppo a72 não consigo activar os 15 Gb de oferta. Ao ligar o 12712 respondem-me que o meu aparelho não está abrangido pela promoção. Algém pode ajudar? Obrigado',
       'boa tarde, há alguma forma de recuperar o código puk? Mudei de telemóvel e não tenho a aplicação Vodafone instalada não dá para entrar e ver o meu PIN e o PUK.  obrigado',
       'boa tarde, há alguma forma de recuperar o código puk? Mudei de telemóvel e não tenho a aplicação Vodafone instalada não dá para entrar e ver o meu PIN e o PUK.  obrigado',
       'Boa tarde, Efectuei recentemente uma renegociação do meu contrato e garantiram-me as mesmas condições que já tinha, mas com alguns upgrades. O que acontece é que passei de uma velocidade de 200/10

In [148]:
post_data2 = {'Data_recolha':post_data['Tempo_recolha'],
              'Categoria':post_data['Categoria'],
              'Empresa':post_data['Empresa'],
              'n': post_data['n'],
              'URL_Post': post_data['URL_Post'],
              'Title_Post': post_data['Theme_Pub'],
              'User_Post': post_data['User_Pub'],
              'Class_User_Post': post_data['Class_User_Pub'],
              'Texto_Post' : post_data['Texto_Pub'],
              'Data_Post' : post_data['Data_Pub'] ,
              'Likes_Post':post_data['Likes_Pub'], 

              'Users' : post_data['User_Conv'],
              'Class_Users': post_data['Class_Conv'], 
              'Data_coments': post_data['Data_Conv'],
              'Coments': post_data['Textos_Conv'], 
              'Likes_Coments': post_data['Likes_Conv']
             }

post_data3 = pd.DataFrame.from_dict(post_data2, orient='index')
base = post_data3.transpose()
base2 = base[base['Texto_Post'].notna()]
base2

,Data_recolha,Categoria,Empresa,n,URL_Post,Title_Post,User_Post,Class_User_Post,Texto_Post,Data_Post,Likes_Post,Users,Class_Users,Data_coments,Coments,Likes_Coments
0,2021-03-18 16:56:41.503305,Telecomunicação,Vodafone,1,https://forum.vodafone.pt/t5/Clube-Viva/N%C3%A...,Não consigo ativar Oferta 15 G,pbsilva76,Novo Utilizador,Boa tarde Após a compra online de uma smarphon...,18-03-2021 03:52 PM,0,pbsilva76,Novo Utilizador,18-03-2021 03:52 PM,Boa tarde Após a compra online de uma smarphon...,0
1,2021-03-18 16:56:41.503305,Telecomunicação,Vodafone,1,https://forum.vodafone.pt/t5/Clube-Viva/N%C3%A...,Não consigo ativar Oferta 15 G,pbsilva76,Novo Utilizador,Boa tarde Após a compra online de uma smarphon...,18-03-2021 03:52 PM,0,Marta_Vodafone,Moderador,18-03-2021 04:44 PM,"Olá @pbsilva76, Já passaram mais de 48h após...",0
2,2021-03-18 16:56:47.604068,Telecomunicação,Vodafone,2,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Recuperar o PUK,marco_vaz,Novo Utilizador,"boa tarde, há alguma forma de recuperar o códi...",18-03-2021 03:50 PM,0,marco_vaz,Novo Utilizador,18-03-2021 03:50 PM,"boa tarde, há alguma forma de recuperar o códi...",0
3,2021-03-18 16:56:47.604068,Telecomunicação,Vodafone,2,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Recuperar o PUK,marco_vaz,Novo Utilizador,"boa tarde, há alguma forma de recuperar o códi...",18-03-2021 03:50 PM,0,Rute_Vodafone,Moderador,18-03-2021 04:53 PM,"Olá @marco_vaz, Para que possamos ajudar, pe...",0
4,2021-03-18 16:56:53.743340,Telecomunicação,Vodafone,3,https://forum.vodafone.pt/t5/Net-Voz/Solicitar...,Solicitar Gravação de contrato por telefone,nrato,Super Visitante,"Boa tarde, Efectuei recentemente uma renegocia...",18-03-2021 03:48 PM,0,nrato,Super Visitante,18-03-2021 03:48 PM,"Boa tarde, Efectuei recentemente uma renegocia...",0
5,2021-03-18 16:56:53.743340,Telecomunicação,Vodafone,3,https://forum.vodafone.pt/t5/Net-Voz/Solicitar...,Solicitar Gravação de contrato por telefone,nrato,Super Visitante,"Boa tarde, Efectuei recentemente uma renegocia...",18-03-2021 03:48 PM,0,Rodolf_Vodafone,Moderador,18-03-2021 04:33 PM,"Olá @nrato, Verificámos que a questão encont...",0
6,2021-03-18 16:57:00.250783,Telecomunicação,Vodafone,4,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Puk perdido,aiguedes,Novo Utilizador,Necessito de ajudar o meu sogro a recuperar o ...,18-03-2021 01:53 PM,0,aiguedes,Novo Utilizador,18-03-2021 01:53 PM,Necessito de ajudar o meu sogro a recuperar o ...,0
7,2021-03-18 16:57:00.250783,Telecomunicação,Vodafone,4,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Puk perdido,aiguedes,Novo Utilizador,Necessito de ajudar o meu sogro a recuperar o ...,18-03-2021 01:53 PM,0,Rute_Vodafone,Moderador,18-03-2021 02:52 PM,"Olá @aiguedes, Respondemos á sua mensagem pr...",0
8,2021-03-18 16:57:06.903112,Telecomunicação,Vodafone,5,https://forum.vodafone.pt/t5/Tv-Servi%C3%A7os/...,Não Renovação de Contrato TV NET VOZ - Erro na...,Barbara51,Novo Utilizador,Venho por este meio solicitar uma revisão da f...,18-03-2021 12:03 PM,0,Barbara51,Novo Utilizador,18-03-2021 12:03 PM,Venho por este meio solicitar uma revisão da f...,0
9,2021-03-18 16:57:06.903112,Telecomunicação,Vodafone,5,https://forum.vodafone.pt/t5/Tv-Servi%C3%A7os/...,Não Renovação de Contrato TV NET VOZ - Erro na...,Barbara51,Novo Utilizador,Venho por este meio solicitar uma revisão da f...,18-03-2021 12:03 PM,0,Rute_Vodafone,Moderador,18-03-2021 12:59 PM,Olá Compreendemos e lamentamos a sua insatis...,0


In [149]:
base

,Data_recolha,Categoria,Empresa,n,URL_Post,Title_Post,User_Post,Class_User_Post,Texto_Post,Data_Post,Likes_Post,Users,Class_Users,Data_coments,Coments,Likes_Coments
0,2021-03-18 16:56:41.503305,Telecomunicação,Vodafone,1,https://forum.vodafone.pt/t5/Clube-Viva/N%C3%A...,Não consigo ativar Oferta 15 G,pbsilva76,Novo Utilizador,Boa tarde Após a compra online de uma smarphon...,18-03-2021 03:52 PM,0,pbsilva76,Novo Utilizador,18-03-2021 03:52 PM,Boa tarde Após a compra online de uma smarphon...,0
1,2021-03-18 16:56:41.503305,Telecomunicação,Vodafone,1,https://forum.vodafone.pt/t5/Clube-Viva/N%C3%A...,Não consigo ativar Oferta 15 G,pbsilva76,Novo Utilizador,Boa tarde Após a compra online de uma smarphon...,18-03-2021 03:52 PM,0,Marta_Vodafone,Moderador,18-03-2021 04:44 PM,"Olá @pbsilva76, Já passaram mais de 48h após...",0
2,2021-03-18 16:56:47.604068,Telecomunicação,Vodafone,2,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Recuperar o PUK,marco_vaz,Novo Utilizador,"boa tarde, há alguma forma de recuperar o códi...",18-03-2021 03:50 PM,0,marco_vaz,Novo Utilizador,18-03-2021 03:50 PM,"boa tarde, há alguma forma de recuperar o códi...",0
3,2021-03-18 16:56:47.604068,Telecomunicação,Vodafone,2,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Recuperar o PUK,marco_vaz,Novo Utilizador,"boa tarde, há alguma forma de recuperar o códi...",18-03-2021 03:50 PM,0,Rute_Vodafone,Moderador,18-03-2021 04:53 PM,"Olá @marco_vaz, Para que possamos ajudar, pe...",0
4,2021-03-18 16:56:53.743340,Telecomunicação,Vodafone,3,https://forum.vodafone.pt/t5/Net-Voz/Solicitar...,Solicitar Gravação de contrato por telefone,nrato,Super Visitante,"Boa tarde, Efectuei recentemente uma renegocia...",18-03-2021 03:48 PM,0,nrato,Super Visitante,18-03-2021 03:48 PM,"Boa tarde, Efectuei recentemente uma renegocia...",0
5,2021-03-18 16:56:53.743340,Telecomunicação,Vodafone,3,https://forum.vodafone.pt/t5/Net-Voz/Solicitar...,Solicitar Gravação de contrato por telefone,nrato,Super Visitante,"Boa tarde, Efectuei recentemente uma renegocia...",18-03-2021 03:48 PM,0,Rodolf_Vodafone,Moderador,18-03-2021 04:33 PM,"Olá @nrato, Verificámos que a questão encont...",0
6,2021-03-18 16:57:00.250783,Telecomunicação,Vodafone,4,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Puk perdido,aiguedes,Novo Utilizador,Necessito de ajudar o meu sogro a recuperar o ...,18-03-2021 01:53 PM,0,aiguedes,Novo Utilizador,18-03-2021 01:53 PM,Necessito de ajudar o meu sogro a recuperar o ...,0
7,2021-03-18 16:57:00.250783,Telecomunicação,Vodafone,4,https://forum.vodafone.pt/t5/Assist%C3%AAncia-...,Puk perdido,aiguedes,Novo Utilizador,Necessito de ajudar o meu sogro a recuperar o ...,18-03-2021 01:53 PM,0,Rute_Vodafone,Moderador,18-03-2021 02:52 PM,"Olá @aiguedes, Respondemos á sua mensagem pr...",0
8,2021-03-18 16:57:06.903112,Telecomunicação,Vodafone,5,https://forum.vodafone.pt/t5/Tv-Servi%C3%A7os/...,Não Renovação de Contrato TV NET VOZ - Erro na...,Barbara51,Novo Utilizador,Venho por este meio solicitar uma revisão da f...,18-03-2021 12:03 PM,0,Barbara51,Novo Utilizador,18-03-2021 12:03 PM,Venho por este meio solicitar uma revisão da f...,0
9,2021-03-18 16:57:06.903112,Telecomunicação,Vodafone,5,https://forum.vodafone.pt/t5/Tv-Servi%C3%A7os/...,Não Renovação de Contrato TV NET VOZ - Erro na...,Barbara51,Novo Utilizador,Venho por este meio solicitar uma revisão da f...,18-03-2021 12:03 PM,0,Rute_Vodafone,Moderador,18-03-2021 12:59 PM,Olá Compreendemos e lamentamos a sua insatis...,0


In [115]:
base.URL_Post[0]

'https://forum.vodafone.pt/t5/Clube-Viva/N%C3%A3o-consigo-ativar-Oferta-15-G/m-p/400538'

### Export

In [150]:
writer = pd.ExcelWriter('BASES/Vodafone_v6.xlsx',
                        engine='xlsxwriter',options={'strings_to_urls': False}) #.format(staName2)
base.to_excel(writer)
writer.close()

#### old

In [100]:
Empresa = ['Vodafone']
titulo=[]
user=[]
data=[]
conteudo=[] 
likes=[]
qt_respostas=[]

for a in soup.findAll('div', attrs={'class':'lia-forum-page-thread-subject-first-page'}):
    name=a.find(attrs={'class':'lia-thread-subject'})
    
    
    titulo.append(name.text) 
    



for a in soup.findAll('div', attrs={'class':'lia-quilt lia-quilt-forum-message lia-quilt-layout-custom-vodafone-message'}):
    name=a.find(attrs={'class':'login-bold'}).text
    name1=a.find(attrs={'class':'DateTime'}).text.replace("\n\u200e"," ").replace("\n"," ").strip()
    name2=a.find(attrs={'class':'lia-message-body-content'}).text.replace("</p><p>"," ").replace("\n"," ").strip()
    name3=a.find('span', attrs={'itemprop':'upvoteCount'}).text.replace("\n","").replace("\t","").strip()



    user.append(name) 
    data.append(name1)
    conteudo.append(name2)
    likes.append(name3)

    


for a in soup.findAll('div', attrs={'class':'lia-text lia-forum-topic-page-reply-count lia-discussion-page-sub-section-header lia-component-reply-count-conditional'}):
    name=a.find(attrs={'class':'lia-hidden'}).text
    
    qt_respostas.append(name)

    
print(user)   
print(titulo)    
print(qt_respostas)


['Paulo-mor', 'BrunoF_Vodafone', 'Paulo-mor', 'Rodolf_Vodafone']
['Problemas com WIFI']
['3']


In [104]:
import numpy as np
forum_Vodafone = pd.DataFrame(np.column_stack([Empresa * len(user),titulo * len(user), user, data, conteudo, likes, qt_respostas * len(user)]), 
                               columns=["Empresa",'titulo','Users', 'Date', 'Comentário', "Likes", "Qt Resposta"])

In [105]:
forum_Vodafone

,Empresa,titulo,Users,Date,Comentário,Likes,Qt Resposta
0,Vodafone,Problemas com WIFI,Paulo-mor,04-12-2020 07:29 PM,"Boa noite,Hoje estive bastante tempo a falar c...",1,3
1,Vodafone,Problemas com WIFI,BrunoF_Vodafone,04-12-2020 08:30 PM,"Olá @Paulo-mor, A situação apenas poderá ser...",0,3
2,Vodafone,Problemas com WIFI,Paulo-mor,04-12-2020 08:36 PM,"Como já percebeu, o número não é Vodafone, poi...",0,3
3,Vodafone,Problemas com WIFI,Rodolf_Vodafone,04-12-2020 09:27 PM,"Olá @Paulo-mor, Compreendemos e lamentamos a...",0,3


In [ ]:
listona = []
listona2 = []
post_url = []
elements = driver.find_elements_by_css_selector('.page-link.lia-link-navigation.lia-custom-event')
for elem in elements:
    urls = elem.get_attribute('href')
    listona.append(urls)                
listona   